In [210]:
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [310]:
### Initializing all the inputs to the code 
DATASET_DIR="./Dataset/"
## CHARACTERS_DIR where each character is present
CHARACTERS_DIR=os.listdir(DATASET_DIR)

## Dimension to which image will resize
RESIZE=(40,40)
##MAX CHARACTER 96
CHARACTERS_TAKEN=3
TRAINING_DATASET="Formatted_Dataset/train.csv"
TESTING_DATASET="Formatted_Dataset/test.csv"
##Training 0.0 to 1.0
TEST_SIZE=0.95
## HIDDEN_LAYER_NEURONS will have the no of neuron in each layer so in this case 3 layers and first layer has 40
## ,2nd 50,3rd 60 
HIDDEN_LAYER_NEURONS=(40,50,60,30,20,)
#activation function values relu,tanh,logistic,identity
ACTIVATION_FUNCTION="relu"
NO_ITERATION=200
MAX_ITERATION=500
#solver lbfgs,sgd,adam
SOLVER="adam"

## Data Preprocessing

In [311]:
## converts 2d to 1d
def dimension_converter(array):
    output=list()
    for i in array:
        for j in i:
            output.append(j)
    return output

In [312]:
## converted dataset will be added fo the file
## labels will <CAPITAL,SMALL>_<alphabet_name>
def save_dataset(data,label):
    global FILE_EXISTS
    global TRAINING_DATASET
    global TESTING_DATASET
    global TEST_SIZE
    if("CAP" in label):     
        label="CAPITAL_"+label.split(".")[-1]
    else:
        label="SMALL_"+label.split(".")[-1]
    data["labels"]=label
    [train,test]=train_test_split(data,shuffle=True,test_size=TEST_SIZE)
    #print(train)
    if(FILE_EXISTS):
        output_train=pd.read_csv(TRAINING_DATASET)
        output_test=pd.read_csv(TESTING_DATASET)
    else:
        output_train=pd.DataFrame()
        output_test=pd.DataFrame()
        FILE_EXISTS=True
    output_train=pd.concat([output_train, train])
    output_test=pd.concat([output_test,test])
    #print(output_test)
    output_train.to_csv(TRAINING_DATASET,index=False)
    output_test.to_csv(TESTING_DATASET,index=False)
    print("For "+label+" Created")

In [313]:
##Reading the image and converting them to the 
FILE_EXISTS=False
COUNT=0
characters_name=os.listdir(DATASET_DIR)

for character in characters_name:
    data1=list()
    dir_loc=DATASET_DIR+character+"/"
    image_names=os.listdir(dir_loc)
    for image in image_names:
        im = Image.open(dir_loc+image)
        im=im.resize(RESIZE)
        pixels = list(im.getdata())
        width, height = im.size
        #print(im.size)
        pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]
        pixels=dimension_converter(pixels)
        data1.append(pixels)
    data1=pd.DataFrame(data)
    save_dataset(data1,character)
    COUNT+=1
    if(COUNT==CHARACTERS_TAKEN):
        break

For CAPITAL_ZETA Created
For SMALL_TAU Created
For CAPITAL_MI Created


## Training the model

In [314]:
def train(SOLVER,ACTIVATION_FUNCTION,NO_ITERATION,MAX_ITERATION):
    print("SOLVER:",SOLVER)
    print("ACTIVATION_FUNCTION",ACTIVATION_FUNCTION)
    print("NO_ITERATION",NO_ITERATION)
    print("MAX_ITERATION",MAX_ITERATION)
    train_data=pd.read_csv(TRAINING_DATASET)
    train_data=train_data.fillna(255)
    train_labels=train_data['labels']
    train_data=train_data.drop(['labels'],axis=1)
    clf = MLPClassifier(solver=SOLVER,
                        hidden_layer_sizes=HIDDEN_LAYER_NEURONS, 
                        random_state=1,activation=ACTIVATION_FUNCTION,
                        n_iter_no_change=NO_ITERATION,
                        max_iter=MAX_ITERATION)
    clf.fit(train_data, train_labels)  
    ## Storing the trained model in file
    pickle.dump(clf,open('models/ann_3_relu.sav', 'wb'))
    file.close()
    test_data=pd.read_csv(TRAINING_DATASET)
    test_data=test_data.fillna(255)
    test_labels=test_data['labels']
    test_data=test_data.drop(['labels'],axis=1)
    a=clf.predict(test_data)
    train_score = accuracy_score(test_labels,a)
    print("score on train data: ", train_score)
    test_data=pd.read_csv(TESTING_DATASET)
    test_data=test_data.fillna(255)
    test_labels=test_data['labels']
    test_data=test_data.drop(['labels'],axis=1)
    a=clf.predict(test_data)
    train_score = accuracy_score(test_labels,a)
    print("score on test data: ", train_score)

    return clf

In [315]:
clf=train(SOLVER,ACTIVATION_FUNCTION,NO_ITERATION,MAX_ITERATION)

SOLVER: adam
ACTIVATION_FUNCTION relu
NO_ITERATION 200
MAX_ITERATION 500
score on train data:  0.9555555555555556
score on test data:  0.30180180180180183


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## Predict the score on trained data

In [295]:
test_data=pd.read_csv(TRAINING_DATASET)
test_data=test_data.fillna(255)
test_labels=test_data['labels']
test_data=test_data.drop(['labels'],axis=1)
a=clf.predict(test_data)
test_score = accuracy_score(test_labels,a)
print("score on train data: ", test_score)

score on train data:  0.9139784946236559


## Predict the score on tested data

In [296]:
test_data=pd.read_csv(TESTING_DATASET)
test_data=test_data.fillna(255)
test_labels=test_data['labels']
test_data=test_data.drop(['labels'],axis=1)
a=clf.predict(test_data)
train_score = accuracy_score(test_labels,a)
print("score on test data: ", train_score)


score on test data:  0.26904761904761904


## Confusion Martix

In [297]:
from sklearn.metrics import confusion_matrix
LABELS=list(set(test_labels))
print(LABELS)
confusion_matrix( test_labels,a,labels=LABELS)

['SMALL_TAU', 'CAPITAL_MI', 'CAPITAL_ZETA']


array([[ 74,  92, 114],
       [101,  65, 114],
       [103,  90,  87]])

In [121]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity

In [228]:
clf.n_outputs_

5

In [230]:
print("s",1,2)

s 1 2
